In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
## Import bigquery package
from google.cloud import bigquery

In [3]:
# Create a "Client" object
client = bigquery.Client()

In [4]:
# Construct a reference to the "hacker_news" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

In [5]:
# Show all the tables in the Stack OverFlow dataset

tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
    print(table.table_id)

In [6]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [7]:
tags_query_2020 = """
                SELECT
                tag_name,
                COUNT(q.id) AS Number_of_Questions,
                AVG (u.reputation) AS avg_reputation,
                SUM (u.reputation) AS sum_reputation
                    FROM 
                        `bigquery-public-data.stackoverflow.tags` AS t
                    INNER JOIN
                        `bigquery-public-data.stackoverflow.posts_questions` AS q
                    ON t.tag_name = q.tags
                    INNER JOIN
                        `bigquery-public-data.stackoverflow.users` AS u
                    ON q.owner_user_id = u.id
                    WHERE q.creation_date >= '2020-01-01'AND q.creation_date <'2021-01-01' # Can be adjusted to your needs!
                    GROUP BY tag_name
                    HAVING Number_of_Questions > 2000 # Adjust to your taste
                    ORDER BY sum_reputation DESC
                    """
tags_2020 = client.query(tags_query_2020).result().to_dataframe()

In [8]:
# Change display format to remove all digits after the decimal point

# pd.options.display.float_format = '{:,.0f}'.format
tags_2020

In [9]:
# Construct a reference to the "badges" table
table_ref = dataset_ref.table("tags")

# API request - fetch the table
table = client.get_table(table_ref)
# Print information on all the columns in the "badges" table in the "Stack Overflow" dataset
table.schema

In [10]:
tags_query_2021 = """
                SELECT
                tag_name,
                COUNT(q.id) AS Number_of_Questions,
                AVG (u.reputation) AS avg_reputation,
                SUM (u.reputation) AS sum_reputation
                    FROM 
                        `bigquery-public-data.stackoverflow.tags` AS t
                    INNER JOIN
                        `bigquery-public-data.stackoverflow.posts_questions` AS q
                    ON t.tag_name = q.tags
                    INNER JOIN
                        `bigquery-public-data.stackoverflow.users` AS u
                    ON q.owner_user_id = u.id
                    WHERE q.creation_date >= '2021-01-01'AND q.creation_date <'2022-01-01' # Can be adjusted to your needs!
                    GROUP BY tag_name
                    HAVING Number_of_Questions > 2000 # Adjust to your taste
                    ORDER BY sum_reputation DESC
                    """
tags_2021 = client.query(tags_query_2021).result().to_dataframe()
tags_2021

In [11]:
view_query_2021 = """
 SELECT 
 SUM(p.view_count) as TypeScript_total_view
     FROM `bigquery-public-data.stackoverflow.posts_questions` AS p
     WHERE tags LIKE '%typescript%'
     AND creation_date >= '2021-01-01'


"""
view_2021 = client.query(view_query_2021).result().to_dataframe()
view_2021

In [12]:
view_javascript_query_2021 = """
 SELECT 
 SUM(p.view_count) as JavaScript_total_view
     FROM `bigquery-public-data.stackoverflow.posts_questions` AS p
     WHERE tags LIKE '%javascript%'
     AND creation_date >= '2021-01-01'


"""
view_2021 = client.query(view_javascript_query_2021).result().to_dataframe()
view_2021

In [13]:
javascript_query_2021 = """
SELECT COUNT(tags) as React FROM
( SELECT 
    tags,view_count
     FROM `bigquery-public-data.stackoverflow.posts_questions` AS p
     WHERE tags LIKE '%typescript%'
     AND creation_date >= '2021-01-01'
     ORDER by view_count DESC
     limit 50) as t 
     WHERE tags LIKE '%rollup%'


"""
view_2021 = client.query(javascript_query_2021).result().to_dataframe()
view_2021

In [14]:
ts_query_2021 = """
 SELECT 
    tags,view_count
     FROM `bigquery-public-data.stackoverflow.posts_questions` AS p
     WHERE tags LIKE '%javascript%'
     AND creation_date >= '2021-01-01'
     ORDER by view_count DESC
     limit 50


"""
view_2021 = client.query(ts_query_2021).result().to_dataframe()
view_2021

In [15]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
stackOverflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="stackoverflow")
js_query_percentage = """
SELECT
  EXTRACT(YEAR FROM creation_date) AS Year,
  COUNT(*) AS Number_of_Questions,
  ROUND(100*SUM(CASE 
  WHEN tags LIKE '%javascript%'THEN 1
    ELSE 0
  END)/COUNT(*),1) AS Percentage_of_Question
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY
  Year
HAVING
  Year > 2008 AND Year < 2022
ORDER BY
  Year;
        """
response1 = stackOverflow.query_to_pandas_safe(js_query_percentage)
response1.head(20)



In [25]:
import bq_helper
import seaborn as sns
import matplotlib.pyplot as plt
import wordcloud
from bq_helper import BigQueryHelper

ax = sns.barplot(x="Year",y="Percentage_of_Question",data=response1,palette="coolwarm").set_title("What is the percentage of questions that have labeded with JavaScript?")

In [16]:
ts_query_percentage = """
SELECT
  EXTRACT(YEAR FROM creation_date) AS Year,
  COUNT(*) AS Number_of_Questions,
  ROUND(100*SUM(CASE 
  WHEN tags LIKE '%typescript%'THEN 1
    ELSE 0
  END)/COUNT(*),1) AS Percentage_of_Question
FROM
  `bigquery-public-data.stackoverflow.posts_questions`
GROUP BY
  Year
HAVING
  Year > 2008 AND Year < 2022
ORDER BY
  Year;
        """
response2 = stackOverflow.query_to_pandas_safe(ts_query_percentage)
response2.head(20)


In [27]:
ax = sns.barplot(x="Year",y="Percentage_of_Question",data=response2,palette="coolwarm").set_title("What is the percentage of questions that have labeded with TypeScript?")